In [205]:
record = {"name":"jon","age":30,"married":True,"rating":7.8}
schema = {"name":str,"age":int,"married":bool,"rating":float,"income":int}

In [206]:
header_fields = []
body = []

for value in record.values():
    if value is None:
        serial_type = constants.SERIAL_RECORD_NULL
        body_bytes = b''

    elif isinstance(value, int):
        serial_type = constants.SERIAL_RECORD_INT
        body_bytes = value.to_bytes(serial_type, 'big', signed=True)

    elif isinstance(value, float):
        serial_type = constants.SERIAL_RECORD_FLOAT
        body_bytes = struct.pack('>d', value)

    elif isinstance(value, str):
        encoded = value.encode('utf-8')
        serial_type = 13 + len(encoded) * 2  # odd => text
        body_bytes = encoded
    else:
        raise TypeError(f"No support for type: {type(value)} yet")
    

    header_fields.append(encode_varint(serial_type))         
    body.append(body_bytes)

# Header size = size of all serial type varints + varint(header_size)
# take header fields and turn them from list to bytestream
header_bytestream = b''.join(header_fields)
# Get total header size to encode
header_size = len(header_bytestream) + 1 #here we are assuming that we won't have more than 128 columns so header_length is always 1 byte long

full_header = encode_varint(header_size) + header_bytestream

row = full_header + b''.join(body)
print(row)

b'\x05\x13\x04\x04\x07jon\x00\x00\x00\x1e\x00\x00\x00\x01@\x1f333333'


In [201]:
row

b'\x05\x13\x04\x04\x07jon\x00\x00\x00\x1e\x00\x00\x00\x01@\x1f333333'

In [203]:
i = 0 

# while i < len(row):

header_size = row[0]


In [204]:
header_size

5

In [96]:
import struct

# The varint algorithm is to stores header_sizes and serial_records in as little space as possible. It transform any int value into multiple 7 bit chunks (usually just 1 is enough)
# We transform the given n into bits and then take chunks of 7 bits (we use the 1st bit to show if there are more 7-bit chunks to read or not)
# taking the number 300 for example: would be 1_0010_1100
# while the values are greater than 128 (8 bits) we are going to continue using multiple 7-bit chunks to stores them
# so we take the first 7 bits --> 010_1100, and append it to chunks_list list of 7 bit chunks, every 7-bit chunk gets a digit
# since there are more bits coming, we add a 1 to the front of the 7 bit --> 1010_1100, is the int=172 --> 0xac (we append this hex to our list)
# now we shift 7 bits to the left from from the original 1_0010_1100 we are now are here 1_0(here)010_1100 which is the same as 000_0010 as a 7 bit chunk
# now that the rest of the 7 bit chunks are just going to be 0s, we know we are done and don't have to add a 1 to the front of this 7 bit chunk, and we just have 0000_0010 which is int = 2 --> 0x02 (we append this hex to our list)

def encode_varint(n: int) -> bytes:
    # can we fit the n into a single 7 bit chunk?
    if n < 128:
        # if yes then just return it as bytes
        return bytes([n])
    # otherwise lets start a list to append multiple 7 bit chunks
    chunks_list = []
    # while we need more than one 7 bit chunk to store the n
    while n >= 128:
        # continue appending the 7 bit chunks and adding 128 (which is the same as adding a 1 to a 7 bit chunk since 1000_0000 in bits is 128)
        chunks_list.append((n & 127) | 128)
        # and then move up 7 bits to the left to get the next 7 bits (same as adjusting n by the number of times its divisible by 128)
        n = n//128
    # we can then append this value (max 255--> 8 bits 1111_1111) to chunks list and process the next 7 bits
    chunks_list.append(n)
    return bytes(reversed(chunks_list))

In [97]:
encode_varint(300)

b'\x02\xac'

In [64]:
my_list.append(127 | 128)

In [86]:
n = 3 


In [87]:
my_list.append((n & 127) | 128)

In [93]:
my_list


[131]

In [92]:
300 & 127

44

In [95]:
hex(172)

'0xac'